# [You Don't Know JavaScript: this & Object Prototypes](https://www.oreilly.com/library/view/you-dont-know/9781491905142/)
Kyle Simpson (O'Reilly)  
Copyright 2014 Getify Solutions Inc.  
978-1-491-90415-2
___

## this or That?
Chapter 1, Page 1 - 9
    
<code>this</code> is a special identifier keyword that's automatically defined in the scope of every function, but it doesn't refer to itself, nor does it refer to its scope.

***Incorrect Assumptions***
1. ***Itself*** - assuming <code>this</code> refers to the function itself
2. ***Its Scope*** - assuming <code>this</code> refers to the function scope 

***Note:*** The following examples are explicitly demonstrations on the incorrect assumptions about <code>this</code>. They do not illistrate on how to use <code>this</code>. That will be demonstrated later on.

### 1. Itself
<code>this</code> doesn't let a function get a reference to itself. 

In [22]:
// DONT DO THIS
// Example of how `this` doesn't refer to itself
(() => {

    function foo(num) {
        console.log('foo: ' + num);
        this.count++;
    }
    
    foo.count = 0; // Accidently creates a global variable with a NaN value
    
    for (let i = 0; i < 10; i++) {
        if (i > 5) {
            foo(i);
        }
    }
    
    console.log(foo.count); // 0?
    
})();

foo: 6
foo: 7
foo: 8
foo: 9
0


A lack of understanding what <code>this</code> means and how it works; developers often fall back to a comfort zone by storing state into lexical scope (like objects). There are better places to store state besides the function object. You'll come to find that passing these state objects as an explicit parameter is often messier than passing around a <code>this</code> context. For example:

In [21]:
// DONT DO THIS
// Example of developers avoid `this` and using an object to store state
(() => {

    function foo(num) {
        console.log('foo: ' + num);
        data.count++;
    }
    
    let data = { // Object storing state
        count: 0
    };
    
    for (let i = 0; i < 10; i++) {
        if (i > 5) {
            foo(i);
        }
    }
    
    console.log(data.count); // 4
    
})();

foo: 6
foo: 7
foo: 8
foo: 9
4


Another way developers avoid <code>this</code> is to reference a function object from inside itself via a lexical identifier (variable) that points at it.

In [24]:
// DONT DO THIS
// Example of using a lexical identifier that points to itself
(() => {

    function foo(num) {
        console.log('foo: ' + num);
        foo.count++; // The lexical identifier
    }
    
    foo.count = 0; // No longer creates a global variable
    
    for (let i = 0; i < 10; i++) {
        if (i > 5) {
            foo(i);
        }
    }
    
    console.log(foo.count); // 4

})();

foo: 6
foo: 7
foo: 8
foo: 9
4


Another way developers use <code>this</code> incorrectly is to force <code>this</code> to point to the function object

In [30]:
// DONT DO THIS
// Example of forcing `this` to point to a function object
(() => {

    function foo(num) {
        console.log('foo: ' + num);
        this.count++;
    }
    
    foo.count = 0; // No longer creates a global variable
    
    for (let i = 0; i < 10; i++) {
        if (i > 5) {
            foo.call(foo, i); // Forcing `this` to point to `foo` by using call()
        }
    }
    
    console.log(foo.count); // 4

})();

foo: 6
foo: 7
foo: 8
foo: 9
4


### 2. Its Scope
<code>this</code> does not, in any way, refer to a function's lexical scope. It is true that internally, scope is kind of like an object with properties for each of the available identifiers. But the scope "object" is not accessible to JavaScript code. It's an inner part of the engine's implementation.

In [43]:
// Example of trying to refer to a functions lexical scope
(()=>{

    function foo() {
        var a = 2;
        this.bar();
    }
    
    function bar() {
        console.log(this.a);
    }
    
    foo();

})();

undefined


You cannot use a <code>this</code> reference to look something up in a lexical scope. Everytime you feel yourself trying to mix lexical scope look-ups with <code>this</code>, remind yourself: there is no bridge.

### What's <code>this</code>?
<code>this</code> is not an author-time binding, but a runtime binding. <code>this</code> has nothing to do with where a function is declared, but has instead everything to do with the manner in which the function was called. When a function is invoked, an activation record, otherwise known as an execution context, is created. This record contains information about where the function was called from (the call stack), how the function was invoked, what parameters were passed etc. One of the properties of this record is the <code>this</code> reference, which will be used for the duration of that functions execution. <code>this</code> is actually a binding that is made when a function is invoked, and what it references is determined entirely by the call-site where the function is called.

___
## <code>this</code> All Makes Sense Now!
Chapter 2, Page 11 - 
    
### Call-Site
***Call-Site*** is the location in code where a function is called (not where it's declared). Finding the call-site is generally "go locate where a function is called from", but it's not always that easy, as certain coding patterns can obscure the true call-site. The call-site we care about is in the invocation before the currently executing function.

In [44]:
(() => {

    function baz() {
        // call-stack is `baz`
        // call-site is global scope
        console.log('baz');
        bar(); // call-site for `bar`
    }
    
    function bar() {
        // call-stack is `baz` --> `bar`
        // call-site is in `baz`
        console.log('bar');
        foo(); // call-site for `foo`
    }
    
    function foo() {
        // call-stack is `baz` --> `bar` --> `foo`
        // call-site is in `bar`
        console.log('foo');
    }
    
    baz(); // call-site for `baz`
    
})();

baz
bar
foo


***Special Note***: take care when analyzing code to find the actual call-site (from the call-stack), because it's the only thing that matters for <code>this</code> binding.